In [1]:
import json
import pandas as pd
from konlpy.tag import Okt # komoran, han, kkma
from jamo import h2j, j2hcj
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from tqdm.notebook import tqdm
import numpy as np

# 데이터 가져오기, 학습 데이터셋으로 만들기

## 인벤에서 수집한 데이터 가져오기

In [151]:
immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)
immoral.columns = ['text']
immoral['immoral'] = True

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2151298477.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  immoral = pd.read_csv(r'data\immoral.txt', sep = 'asssssssss', header = None)


In [152]:
clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)
clean.columns = ['text']
clean['immoral'] = False

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2827705178.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  clean = pd.read_csv(r'data\clean.txt', sep = 'asssssssss', header = None)


## op.gg 데이터 크롤링

In [31]:
import requests
from bs4 import BeautifulSoup
import json
import time

In [ ]:
lii = []
headers = {
    'Referer':'https://www.op.gg/',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
}
for chm in champ:
    n = 1
    while 1:
        url = f'https://op.gg/api/v1.0/internal/bypass/champions/{chm}/comments?&sort=popular&page={n}&limit=100&hl=ko_KR&is_latest_version=false'
        n += 1
        resp = requests.get(url, headers = headers)
        dict_ch = json.loads(resp.text)
        if len(dict_ch['data']) == 0:
            break
        
        for dat in dict_ch['data']:
            lii.append(dat['content'].replace('\n', ''))
        time.sleep(5)
    
    print(chm)

In [75]:
df_lii = pd.DataFrame(lii, columns = ['text'])
df_lii['immoral'] = ''
df_lii

,text,immoral
0,얘 와맆에 곰돌이가 에어본 시키는 거 생겼던데 여기에도 생기나,
1,자기고향 찾아오네,
2,몇달만의 고향이냐,
3,점멸 있을 때 한정 1티어,
4,스킬 사거리 표시 없이 하면 암걸리는 챔,
...,...,...
7968,의외로 꼴짤이 별로 없음,
7969,나피리 근데 초반 라인전도 약하고 카운터도 많음.그리고 돌진기가 다른 암살자와 다르...,
7970,0/6/0 야스오:야이 개쌔꺄 이게 챔이냐 야발4/0/2 나피리:개 맞는데0/8/0...,
7971,얘 한테 지면 사람아닌래끼다버프좀 하고 성능좀 버프히라못 쓸 정도다,


### op.gg 데이터 직접 라벨링, 라벨링 된 csv 가져오기

In [81]:
# df_lii.to_csv(r'data/champions.csv', index = False)
df_lii = pd.read_csv(r'data/champions.csv')

In [195]:
champions = pd.read_csv(r'data\champions.csv')
champions.columns = ['text','immoral']
champions.loc[champions['immoral'] == 0, 'immoral'] = False
champions.loc[champions['immoral'] == 1, 'immoral'] = True
champions = champions.fillna(False)
champions

C:\Users\user\AppData\Local\Temp\ipykernel_10380\2204739627.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  champions.loc[champions['immoral'] == 0, 'immoral'] = False


,text,immoral
0,얘 와맆에 곰돌이가 에어본 시키는 거 생겼던데 여기에도 생기나,False
1,자기고향 찾아오네,False
2,몇달만의 고향이냐,False
3,점멸 있을 때 한정 1티어,False
4,스킬 사거리 표시 없이 하면 암걸리는 챔,False
...,...,...
6458,클레드로 굶드라랑 월식 못 가게 너프먹는다니까 진짜 ㅈ같아서 겜 못해먹겠네 월식은 ...,True
6459,진짜 개사기임아니 협곡말고 롤체 학살자 클레드요,False
6460,진짜 태불방 롤백 시킬거 아니면 클레드는 건들면 안되지 ㅋㅋㅋ 선넘네 진짜,False
6461,"""바텀에서 겨우 더블킬을 했다.""""나팔소리가 들렸다.""""여름이었다.""(제압되었습니다!)",False


## Ai hub 데이터 가져오기

In [154]:
# ai hub train set
li = []
for i in range(1, 6):
    with open(r'.\data\train\labled\talksets-train-' + str(i) + '.json') as f:
        data = json.load(f)
    for d in data:
        for s in d['sentences']:
            li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                    'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                    'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
            
train = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])
train['text'] = train['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

train['immoral'] = (train['SEXUAL'] == True) | (train['ABUSE'] == True)
train_set = train[['text', 'immoral']]

In [156]:
# ai hub test set
li = []
with open(r'.\data\valid\labled\talksets-train-6.json') as f:
    data = json.load(f)
for d in data:
    for s in d['sentences']:
        li.append([s['origin_text'], s['is_immoral'], s['intensity'], 
                'CENSURE' in s['types'], 'HATE' in s['types'], 'DISCRIMINATION' in s['types'], 
                'SEXUAL' in s['types'], 'ABUSE' in s['types'], 'VIOLENCE' in s['types'], 'CRIME' in s['types']])
test = pd.DataFrame(li, columns=['text', 'immoral', 'intensity', 'CENSURE', 'HATE', 'DISCRIMINATION', 'SEXUAL', 'ABUSE', 'VIOLENCE', 'CRIME'])

test['text'] = test['text'].apply(lambda x : re.sub('#@.+?#', '', x.replace('#@#', '')))

test['immoral'] = (test['SEXUAL'] == True) | (test['ABUSE'] == True)
test_set = test[['text', 'immoral']]

## badwords 가져오기
https://github.com/organization/Gentleman/blob/master/resources/badwords.json

In [184]:
# badwords, train
with open(r".\data\train\labled\badwords.json") as f:
    data = json.load(f)

badwords = pd.DataFrame(data['badwords'], columns= ['text'])
badwords['immoral'] = True

## 가져온 데이터 train/test 나눠서 합치기

In [166]:
im_sl = int(len(immoral) * 0.8)
cl_sl = int(len(clean) * 0.8)
cp_sl = int(len(champions) * 0.8)
bw_sl = int(len(badwords) * 0.8)

In [211]:
train = pd.concat([train_set, immoral[:im_sl], clean[:cl_sl], badwords[:bw_sl], champions[:cp_sl]])
test = pd.concat([test_set, immoral[im_sl:], clean[cl_sl:], badwords[bw_sl:], champions[cp_sl:]])
train['immoral'] = train['immoral'].apply(lambda x : 1 if x == True else 0)
test['immoral'] = test['immoral'].apply(lambda x : 1 if x == True else 0)
train.shape, test.shape

((369610, 2), (46830, 2))

# 데이터 전처리


In [48]:
# train.to_csv(r'data/train_final.csv', index = False)
# test.to_csv(r'data/test_final.csv', index = False)
# train = pd.read_csv(r'data/train_aug2.csv')
# test = pd.read_csv(r'data/test_final.csv')
# train = pd.read_csv(r'data/train_aug.csv')
# test = pd.read_csv(r'data/test_aug.csv')

test = pd.read_csv(r'data/champions_t.csv')
train = train.dropna()
test = test.dropna()
train.shape, test.shape

((778841, 3), (247, 2))

## 데이터 자모음으로 쪼개기

In [49]:
train['text'] = train['text'].apply(lambda x : j2hcj(h2j(x)))
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))

In [4]:
# swap rows
train = train.sample(frac = 1).reset_index(drop=True)

In [5]:
def textToList(text):
    li = []
    for t in text:
        li.append(list(t.strip()))
    return li

In [50]:
# 자모음 쪼갠 텍스트를 한글자씩 리스트로 만들어서 데이터셋 만든다
def textToList(text):
    li = []
    for t in text:
        li.append(list(t))
    return li

train_list = textToList(train['text'])
train_label = train['immoral']

test_list = textToList(test['text'])
test_label = test['immoral']

# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [7]:
# 문장 최대 길이 정해야되니까 문장 길이 분포가 얼마나 되어있는지 확인한다
# li_len = []
# for t in train_list:
#     li_len.append(len(t))
# import seaborn as sns
# sns.set_theme(style="darkgrid")
# sns.histplot(li_len)
## 100으로 자르면 문장 잘리는 아이들 비율이 2%

In [51]:
tokenizer = Tokenizer(num_words=110)
tokenizer.fit_on_texts(train_list) # 단어 인덱스 구축
text_sequences = tokenizer.texts_to_sequences(train_list) # 문자열 -> 인덱스 리스트
                                                            # '나는 천재다 나는 멋있다' -> [1, 2, 1, 3]
# train data로 fit 시켜준 tokenizer로 test data도 인덱스로 변경
text_sequences_test = tokenizer.texts_to_sequences(test_list)

word_vocab = tokenizer.word_index # 딕셔너리 형태
print("전체 단어 개수: ", len(word_vocab)) # 전체 단어 개수 확인

전체 단어 개수:  166


In [52]:
MAX_SEQUENCE_LENGTH = 100 # 문장 최대 길이

X_train = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_train = np.array(train_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

print('Shape of train input data tensor:', X_train.shape) # 리뷰 데이터의 형태 확인
print('Shape of train label tensor:', y_train.shape) # 감정 데이터 형태 확인
print('Shape of test input data tensor:', X_test.shape) # 리뷰 데이터의 형태 확인
print('Shape of test label tensor:', y_test.shape) # 감정 데이터 형태 확인

Shape of train input data tensor: (778841, 100)
Shape of train label tensor: (778841,)
Shape of test input data tensor: (247, 100)
Shape of test label tensor: (247,)


In [9]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Flatten, Dropout
from tensorflow.keras import layers, models

embedding_dim = 50 # 임베딩 벡터의 차원
dropout_ratio = 0.3 # 드롭아웃 비율
num_filters = 256 # 커널의 수
kernel_size = 3 # 커널의 크기
hidden_units = 128 # 뉴런의 수

# model = Sequential()
# model.add(Embedding(len(word_vocab)+1, embedding_dim))
# model.add(Conv1D(256, kernel_size, padding='valid', activation='relu'))
# model.add(Conv1D(256, kernel_size, padding='valid', activation='relu'))
# model.add(Conv1D(num_filters, kernel_size, padding='valid', activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(hidden_units, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

vocab_size = 30000
word_vector_dim = 32

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(word_vocab)+1, word_vector_dim, input_shape=(None,)))
model.add(tf.keras.layers.Conv1D(16, 5, activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(5))
model.add(tf.keras.layers.Conv1D(16, 5, activation='relu'))
model.add(tf.keras.layers.GlobalMaxPooling1D())
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics=['accuracy',
                                                                            tf.keras.metrics.TrueNegatives(name='true_negatives'),
                                                                            tf.keras.metrics.TruePositives(name='true_positives'),
                                                                            tf.keras.metrics.FalseNegatives(name='false_negatives'),
                                                                            tf.keras.metrics.FalsePositives(name='false_positives')])
# 이진 분류이므로 손실함수는 binary_crossentropy 사용, 에폭마다 정확도를 보기 위해 accuracy 적용
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          5344      
                                                                 
 conv1d (Conv1D)             (None, None, 16)          2576      
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 16)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 16)          1296      
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 16)                0

In [12]:
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.2, # new_lr = lr * factor.
    patience=2,
    cooldown=2, # number of epochs to wait before resuming normal operation after lr has been reduced.  
    min_lr=0
)
model.fit(X_train, y_train, epochs= 100, batch_size = 100, validation_split=0.2, callbacks=[early, reduce_lr])

model.save('conv_red.h5')

Epoch 1/100
6231/6231 [==============================] - 129s 21ms/step - loss: 0.1417 - accuracy: 0.8006 - true_negatives: 206050.0000 - true_positives: 292768.0000 - false_negatives: 63472.0000 - false_positives: 60782.0000 - val_loss: 0.1479 - val_accuracy: 0.7915 - val_true_negatives: 55895.0000 - val_true_positives: 67401.0000 - val_false_negatives: 21677.0000 - val_false_positives: 10796.0000 - lr: 4.0000e-05
Epoch 2/100
6231/6231 [==============================] - 127s 20ms/step - loss: 0.1416 - accuracy: 0.8008 - true_negatives: 206213.0000 - true_positives: 292733.0000 - false_negatives: 63507.0000 - false_positives: 60619.0000 - val_loss: 0.1495 - val_accuracy: 0.7888 - val_true_negatives: 56769.0000 - val_true_positives: 66095.0000 - val_false_negatives: 22983.0000 - val_false_positives: 9922.0000 - lr: 4.0000e-05
Epoch 3/100
6231/6231 [==============================] - 121s 19ms/step - loss: 0.1416 - accuracy: 0.8008 - true_negatives: 206393.0000 - true_positives: 292571.00

In [55]:
pred_y_train = model.predict(X_train)
print(model.evaluate(X_train, y_train))

pred_y_test = model.predict(X_test)
print(model.evaluate(X_test, y_test))

8/8 [==============================] - 0s 1ms/step - loss: 0.1311 - accuracy: 0.8219 - true_negatives: 4.0000 - true_positives: 199.0000 - false_negatives: 36.0000 - false_positives: 8.0000  
[0.1311338096857071, 0.8218623399734497, 4.0, 199.0, 36.0, 8.0]


In [16]:
train_org = pd.read_csv(r'data/train_aug.csv')
train_org = train_org.dropna()
test_org = pd.read_csv(r'data/test_aug.csv')
train_org['pred'] = np.where(pred_y_train > 0.5, 1, 0)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)


In [61]:

test = pd.read_csv(r'data/champions_t.csv')
test_org = pd.read_csv(r'data/champions_t.csv')
test = test.fillna(0)
test_org = test_org.fillna(0)
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))
test_list = textToList(test['text'])
test_label = test['immoral']
text_sequences_test = tokenizer.texts_to_sequences(test_list)
X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

pred_y_test = model2.predict(X_test)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)
print(classification_report(test_org['immoral'], test_org['pred']))

29/29 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         0.0       0.93      0.67      0.78       688
         1.0       0.47      0.85      0.60       235

    accuracy                           0.72       923
   macro avg       0.70      0.76      0.69       923
weighted avg       0.81      0.72      0.74       923



In [72]:

test = pd.read_csv(r"C:\Users\llljw\Downloads\clean.txt", sep = 'aaaaaaaaaaaaaaaaaaaaaaaa')
test.columns = ['text']
test['immoral'] = 0
test_org = test.copy()
test = test.fillna(0)
test_org = test_org.fillna(0)
test['text'] = test['text'].apply(lambda x : j2hcj(h2j(x)))
test_list = textToList(test['text'])
test_label = test['immoral']
text_sequences_test = tokenizer.texts_to_sequences(test_list)
X_test = pad_sequences(text_sequences_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre') # 문장의 길이가 100 글자가 넘어가면 자르고, 모자르면 0으로 채워 넣는다.
y_test = np.array(test_label) # 각 리뷰의 감정을 넘파이 배열로 만든다.

pred_y_test = model2.predict(X_test)
test_org['pred'] = np.where(pred_y_test > 0.5, 1, 0)
print(classification_report(test_org['immoral'], test_org['pred']))

9/9 [==============================] - 0s 875us/step
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       269
           1       0.00      0.00      0.00         0

    accuracy                           0.91       269
   macro avg       0.50      0.46      0.48       269
weighted avg       1.00      0.91      0.95       269



C:\Users\llljw\AppData\Local\Temp\ipykernel_17788\2565546387.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv(r"C:\Users\llljw\Downloads\clean.txt", sep = 'aaaaaaaaaaaaaaaaaaaaaaaa')
c:\Users\llljw\anaconda3\envs\mzpark\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\llljw\anaconda3\envs\mzpark\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [71]:
test_org

,text,immoral,pred
0,캡쳐해서 인벤에 박제해야지,1,0
1,베인이 욕심부려서 못밀은거잖아,1,0
2,어디서 내가 욕심을 부렸는데?,1,0
3,내가 핑을 쳤어 뭘했어?,1,0
4,미니언 먹는다고 암 말 안했는데,1,0
...,...,...,...
264,역시 팅ㅇ기신거였네요,1,0
265,화이팅,1,0
266,ㅎㅇㅎㅇ,1,0
267,고소장ㄱㄱ,1,0


In [57]:
test_org

,text,immoral
0,사미라는 황밸인거같은데 무대는 리워크좀해라최소 치명타 50%고정정도만 해도 쓸만하지않음?,NaN
1,드락 너프 씨게 먹었던데 이제 징수의 총 -> 무한의 대검 가야 되나,NaN
2,사미라로 드락 별로라는 소리가 있는데 드락은 하위템이 롱소드 박박이라 가는거임 무대...,NaN
3,서폿이 잘하면 무조건 이기는 챔프,NaN
4,드락 왜가는지 이해가 안됨;; 드락보다 무대가 훨씬 난데,NaN
...,...,...
918,클레드로 굶드라랑 월식 못 가게 너프먹는다니까 진짜 ㅈ같아서 겜 못해먹겠네 월식은 ...,1.0
919,진짜 개사기임아니 협곡말고 롤체 학살자 클레드요,0.0
920,진짜 태불방 롤백 시킬거 아니면 클레드는 건들면 안되지 ㅋㅋㅋ 선넘네 진짜,0.0
921,"""바텀에서 겨우 더블킬을 했다.""""나팔소리가 들렸다.""""여름이었다.""(제압되었습니다!)",0.0


In [19]:
from sklearn.metrics import classification_report
print(classification_report(train_org['immoral'], train_org['pred']))
print(classification_report(test_org['immoral'], test_org['pred']))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77    333523
           1       0.24      0.21      0.23    108258

    accuracy                           0.65    441781
   macro avg       0.50      0.50      0.50    441781
weighted avg       0.63      0.65      0.64    441781

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     41924
           1       0.81      0.65      0.72     14718

    accuracy                           0.87     56642
   macro avg       0.84      0.80      0.82     56642
weighted avg       0.86      0.87      0.86     56642



In [17]:
from sklearn.metrics import classification_report
model1 = tf.keras.models.load_model('cnn.h5')
model2 = tf.keras.models.load_model('conv_red.h5')
model3 = tf.keras.models.load_model('conv2.h5')
model4 = tf.keras.models.load_model('conv3.h5')

pred = []
models = [model1, model2, model3, model4]
for m in models:
    pred_y_test = m.predict(X_test)
    pred.append(pred_y_test.reshape(-1,))
    print(m.evaluate(X_test, y_test))
    pred_y_test = np.where(pred_y_test > 0.5, 1, 0)
    print(classification_report(y_test, pred_y_test.reshape(-1,)))
    print('*' * 30)


1464/1464 [==============================] - 8s 5ms/step - loss: 1.2017 - accuracy: 0.4003
[1.2017261981964111, 0.400341659784317]
              precision    recall  f1-score   support

           0       0.93      0.36      0.52     41924
           1       0.12      0.78      0.21      4906

    accuracy                           0.40     46830
   macro avg       0.53      0.57      0.36     46830
weighted avg       0.85      0.40      0.48     46830

******************************
1464/1464 [==============================] - 2s 1ms/step - loss: 0.1350 - accuracy: 0.8215 - true_negatives: 35158.0000 - true_positives: 3312.0000 - false_negatives: 1594.0000 - false_positives: 6766.0000
[0.1350131332874298, 0.8214819431304932, 35158.0, 3312.0, 1594.0, 6766.0]
              precision    recall  f1-score   support

           0       0.96      0.84      0.89     41924
           1       0.33      0.68      0.44      4906

    accuracy                           0.82     46830
   macro avg 

In [20]:
df = pd.DataFrame(pred)

In [38]:
pred[0] = pred[0] * 0.4
pred[1] = pred[1] * 0.82
pred[2] = pred[2] * 0.14
pred[3] = pred[3] * 0.42

In [41]:
df = pd.DataFrame(pred)
mn = df.sum(axis = 0).values
mn = np.where(mn > 0.9, 1, 0)
print(classification_report(y_test, mn))

              precision    recall  f1-score   support

           0       0.96      0.61      0.75     41924
           1       0.19      0.80      0.31      4906

    accuracy                           0.63     46830
   macro avg       0.58      0.70      0.53     46830
weighted avg       0.88      0.63      0.70     46830



In [31]:
mn

array([0, 0, 1, ..., 1, 1, 0])

In [42]:
!pip install lime

     ---------------------------------------- 0.0/275.7 kB ? eta -:--:--
     ------------------------------------- 275.7/275.7 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for scikit-image>=0.12 from https://files.pythonhosted.org/packages/f3/93/65601f7577d6fd49ec23bf8fb58c04d8170b06a1544452ae2ea9f59bf11f/scikit_image-0.21.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for imageio>=2.27 from https://files.pythonhosted.org/packages/eb/21/662994d78d8623055f8ffa91838e28f04b2a34bd5d8d6dbc6c7573285ed6/imageio-2.31.3-py3-none-any.whl.metadata
  Obtaining dependency information for tifffile>=2022.8.12 from https://files.pythonhosted.org/packages/12/3e/89513f44a10c625121b7d5bc54390d7ac7f2c92a19755c052888febf9730/tifffile-2023.8.30-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
     ----------------------

In [45]:
from lime import lime_tabular

explainer = lime_tabular.LimeTabularExplainer(X_train, mode="classification")

idx = 3

print("Prediction : ", model2.predict(X_test[idx].reshape(1,-1)))
print("Actual :     ", y_test[idx])

explanation = explainer.explain_instance(X_test[idx], model2, labels=(0,) ,num_samples=5000)

1/1 [==============================] - 0s 15ms/step
Prediction :  [[0.09567054]]
Actual :      0


AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [47]:
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(model2) # 벡터화 + 나이브베이즈 모델

# 파이프라인을 이용해 데이터 하나에 대한 (벡터화+예측확률)을 반환하자. (fit, transform 과정이 불필요)
predict_classes=pipe.predict_proba([x_test[0]]).round(3)[0]

print(predict_classes)

AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer=LimeTextExplainer(class_names=[0, 1])
exp=explainer.explain_instance(X_test.data[0], # 첫 번째 텍스트 데이터.
                              pipe.predict_proba,# 텍스트를 입력으로 받고,(벡터화를 거쳐) 카테고리 별 확률을 반환한다.
                              top_labels=1) # 가장 확률이 높은 클래스만 보여준다.

In [ ]:
exp.show_in_notebook(text=newsgroups_test.data[0])